### Prozesse aus 1 für neue Monate wiederholen

In [3]:
## Bibliotheken importieren
import pandas as pd
import os

In [20]:
## Top 1000-Liste von switch.ch importieren
directory = 'data/import/'
all_data = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        date = filename[8:14]
        urls_import = pd.read_csv(directory + filename, header=0, names=["domainname", date])
        all_data = pd.concat([all_data, urls_import.set_index('domainname')], axis=1, sort=False)

In [24]:
## Datensatz transponieren
data_tp = all_data.T

In [27]:
## Datetime-Index erstellen
data_tp["date"] = pd.to_datetime(data_tp.index, format="%Y%m")
data_tp = data_tp.set_index("date")

In [29]:
## zusammengefasste und formatierte Daten zwischenspeichern ##
data_tp.to_pickle('data/all_data_update.pkl')

In [30]:
## Und gleich wieder laden:
data_tp = pd.read_pickle('data/all_data_update.pkl')

#### Domains mit Hilfe von externer Webseite klassifizieren:

In [61]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import json
import requests
import time
import re
from pathlib import Path

In [58]:
# Vergleich mit bereits klassifizierten Domains:
data_old = pd.read_pickle('data/all_data.pkl')
new_domains = set(data_tp.columns.values.tolist()).difference(set(data_old.columns.values.tolist()))
len(new_domains)

56

In [62]:
## Neue Domains klassifizieren
driver = webdriver.Chrome("./tools/chromedriver")
url = "https://sitereview.bluecoat.com/"
driver.get(url)
driver.implicitly_wait(100)
url_categories = []
for url in new_domains:
    url_entry = [url]
    time.sleep(1)
    driver.find_element_by_id("txtSearch").send_keys(url)
    time.sleep(1)
    driver.find_element_by_id("btnLookupSubmit").click()
    time.sleep(1)
    category_entry = []
    categories = driver.find_elements_by_class_name("clickable-category")
    if categories:
        for category in categories:
            category_entry.append(category.text)
    url_entry.append(category_entry)
    url_categories.append(url_entry)
    try:
        driver.find_element_by_xpath("/html[1]/body[1]/app-root[1]/div[1]/section[1]/div[1]/ng-component[1]/form[1]/p[1]/a[1]").click()
    except:
        driver.find_element_by_id("btnOk").click()
        driver.find_element_by_xpath("/html[1]/body[1]/app-root[1]/div[1]/section[1]/div[1]/ng-component[1]/form[1]/p[1]/a[1]").click()
driver.quit()

In [65]:
url_categories

[['calyps.ch', ['Unternehmen/Wirtschaft (Business/Economy)']],
 ['ibllc.ch', []],
 ['easyzone.ch', []],
 ['tineonet.ch', []],
 ['cardcenter.ch', ['Finanzdienstleistungen (Financial Services)']],
 ['happyfashion.ch', ['Einkaufen (Shopping)']],
 ['bausch-lomb.ch', ['Gesundheit (Health)']],
 ['goeuro.ch', ['Reisen (Travel)']],
 ['solique.ch', ['Unternehmen/Wirtschaft (Business/Economy)']],
 ['chregister.ch', ['Unternehmen/Wirtschaft (Business/Economy)']],
 ['eta.ch', ['Unternehmen/Wirtschaft (Business/Economy)']],
 ['centralnet.ch', ['Technik/Internet (Technology/Internet)']],
 ['leparking.ch', ['Fahrzeuge (Vehicles)']],
 ['ddnss.ch', ['Dynamischer DNS-Host (Dynamic DNS Host)']],
 ['ilef.ch', ['Regierung/Recht (Government/Legal)']],
 ['edificom.ch', ['Unternehmen/Wirtschaft (Business/Economy)']],
 ['usb.ch', ['Gesundheit (Health)']],
 ['video2000.ch', ['Referenz (Reference)']],
 ['web-alm.ch', ['Technik/Internet (Technology/Internet)']],
 ['ipdinstitute.ch', ['Bildung (Education)']],
 ['b

In [66]:
## DF erstellen
url_categories_df = pd.DataFrame(columns=['url', 'n_categories'])
for url in url_categories:
    n_categories = 0
    if (len(url[1]) > 0):
        for idx, category in enumerate(url[1]):
            if category not in url_categories_df.columns.values:
                url_categories_df[category] = ""
            n_categories = idx + 1
    x = pd.DataFrame([[False] * len(url_categories_df.columns.values)], columns = url_categories_df.columns.values)
    x['url'] = url[0]
    x['n_categories'] = n_categories
    if len(url[1]) > 0:
        for idx, category in enumerate(url[1]):
            x[category] = True
    url_categories_df = url_categories_df.append(x)
    
    

In [67]:
url_categories_df

,url,n_categories,Unternehmen/Wirtschaft (Business/Economy),Finanzdienstleistungen (Financial Services),Einkaufen (Shopping),Gesundheit (Health),Reisen (Travel),Technik/Internet (Technology/Internet),Fahrzeuge (Vehicles),Dynamischer DNS-Host (Dynamic DNS Host),Regierung/Recht (Government/Legal),Referenz (Reference),Bildung (Education),Glücksspiel (Gambling),Verdächtig (Suspicious),Nachrichten/Medien (News/Media),Spam (Spam),Software-Downloads (Software Downloads),Unterhaltung (Entertainment)
0,calyps.ch,1,True,,,,,,,,,,,,,,,,
0,ibllc.ch,0,False,,,,,,,,,,,,,,,,
0,easyzone.ch,0,False,,,,,,,,,,,,,,,,
0,tineonet.ch,0,False,,,,,,,,,,,,,,,,
0,cardcenter.ch,1,False,True,,,,,,,,,,,,,,,
0,happyfashion.ch,1,False,False,True,,,,,,,,,,,,,,
0,bausch-lomb.ch,1,False,False,False,True,,,,,,,,,,,,,
0,goeuro.ch,1,False,False,False,False,True,,,,,,,,,,,,
0,solique.ch,1,True,False,False,False,False,,,,,,,,,,,,
0,chregister.ch,1,True,False,False,False,False,,,,,,,,,,,,


In [68]:
## URL-Kategorien zwischenspeichern ##
url_categories_df.to_pickle('data/url_categories_update.pkl')

In [4]:
## und URL-Kategorien gleich wieder einlesen:
url_categories_df = pd.read_pickle('data/url_categories_update.pkl')

In [6]:
### "neue" Bösartige URLs ermitteln
url_categories_df.url[(url_categories_df['Verdächtig (Suspicious)'] == True)]


0    securewebservices.ch
0     onlinemoviewatch.ch
Name: url, dtype: object